In [58]:
import os
from glob import glob

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Document
from llama_index.llms.ollama import Ollama

# Initialize Ollama LLM
llm_s = Ollama(model="codegemma:latest", request_timeout=10000, temperature=0)
llm_g = Ollama(model="codestral:latest", request_timeout=10000)

dependecy_folders = [
    'Model',
    "Repository"
]

base_directory = "C:\\GitHub\\microservices-sample\\microservices\\src\microservices\\CartMicroservice"

source_files = [
    "C:\\GitHub\\microservices-sample\\microservices\\src\microservices\\CartMicroservice\\Controllers\\CartController.cs"
]

test_target_directory = "C:\\GitHub\\microservices-sample\\microservices\\tests\\CartMicroservice.UnitTests"

file_content = ""

def strip_code(resp, start_string, end_string):
    """
    Removes all content including start_string from the start,
    and all content including end_string from the end of resp.text.
    Returns the cleaned code as a string.
    """
    text = resp.text
    start_idx = text.find(start_string)
    if start_idx != -1:
        text = text[start_idx + len(start_string):]
    end_idx = text.rfind(end_string)
    if end_idx != -1:
        text = text[:end_idx]
    return text.strip()


In [59]:
import json
import os

#llm =  Ollama(model="codestral:latest", request_timeout=10000)
unit_test_cases = []

dependecy_folders = [
    'Model',
    "Repository"
]

def get_all_dependencies(base_directory, dependecy_folders):
    dependencies_content = []
    for folder in dependecy_folders:
        folder_path = os.path.join(base_directory, folder)
        if os.path.isdir(folder_path):
            for file in glob(os.path.join(folder_path, '*')):
                if os.path.isfile(file):
                    with open(file, 'r', encoding='utf-8') as dep_file:
                        dependencies_content.append(dep_file.read())
    return "\n----------\n".join(dependencies_content)

code_dependency = get_all_dependencies(base_directory, dependecy_folders);

# loop over source_files and print the file content 
for file_path in source_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        file_content = f.read( )

        unit_test_scenario_prompt = f"""
        You are given the following C# source file content:

        {file_content}

        The following are the contents of all relevant dependency files (models, repositories, etc.):

        {code_dependency}

        Analyze this code and all dependencies above to generate a list of possible unit test cases for the methods in this file.
        For each test case, make sure that all input parameters (including complex types and nested objects) are given example values for initialization, based on the definitions in the dependency files.

        Return your answer strictly in the following JSON format:

        [
            {{
            "method": "<method_name>",
            "description": "<short description of the test case>",
            "inputs": {{ "<parameter1>": "<example_value>", ... }},
            "expected_result": "<expected outcome or behavior>"
            }},
            ...
        ]

        Do not include any explanations or text outside the JSON array.
        """

        resp = llm_s.complete ( unit_test_scenario_prompt )
        clean_json = strip_code( resp, "```json", "```")

        print ( clean_json )
        
        # Extract the text from the response and load as JSON
        unit_test_cases = json.loads(clean_json)
    
        print(unit_test_cases)

        # Get the base filename without extension
        input_filename = os.path.splitext(os.path.basename(file_path))[0]
        # Create the test filename by adding 'Test' before the extension
        test_filename = f"{input_filename}GenAITest.cs"
        test_file_path = os.path.join(test_target_directory, test_filename)

        code_dependency = get_all_dependencies(base_directory, dependecy_folders);


        # Create a prompt to generate unit test code based on the JSON test cases
        unit_test_code_prompt = f"""
        You are given the following list of unit test cases in JSON format:

        {json.dumps(unit_test_cases, indent=4)}

        The original C# file content is:

        {file_content}

        The following are the contents of all relevant dependency files (models, repositories, etc.):

        {code_dependency}

        Analyze all dependencies above to:
        - Initialize all required objects and mocks in the generated unit tests, no object should be left for user input for initalization
        - Import all relevant dependencies, namespaces, and using statements so the test file compiles and runs.
        - Use Moq to mock dependencies as needed.
        - Use clear and descriptive test method names.
        - Do not include explanations or comments outside the code.
        - The test class should be named '{input_filename}Test'.
        - The namespace should match the original file's namespace, with '.UnitTests' appended.

        Return only the complete C# code for the test file.
        """

        # Get the generated unit test code from the LLM
        unit_test_code_resp = llm_g.complete(unit_test_code_prompt)
        unit_test_code = unit_test_code_resp.text

        # Clean the code block markers from the generated code
        code_start = unit_test_code.find('```csharp')
        if code_start != -1:
            unit_test_code = unit_test_code[code_start + len('```csharp'):]
        code_end = unit_test_code.rfind('```')
        if code_end != -1:
            unit_test_code = unit_test_code[:code_end]
        unit_test_code = unit_test_code.strip()

        # Save the cleaned code to the test file
        with open(test_file_path, 'w', encoding='utf-8') as test_file:
            test_file.write(unit_test_code)

        print(f"Unit test file generated at: {test_file_path}")




[
  {
    "method": "GetCartItems",
    "description": "Should return a list of cart items for a given user",
    "inputs": { "userId": "user123" },
    "expected_result": [
      { "catalogItemId": "item1", "name": "Product A", "price": 10.99, "quantity": 2 },
      { "catalogItemId": "item2", "name": "Product B", "price": 5.99, "quantity": 1 }
    ]
  },
  {
    "method": "InsertCartItem",
    "description": "Should add a new cart item to the user's cart",
    "inputs": {
      "userId": "user123",
      "cartItem": {
        "catalogItemId": "item3",
        "name": "Product C",
        "price": 7.99,
        "quantity": 1
      }
    },
    "expected_result": null
  },
  {
    "method": "UpdateCartItem",
    "description": "Should update an existing cart item",
    "inputs": {
      "userId": "user123",
      "cartItem": {
        "catalogItemId": "item1",
        "name": "Product A",
        "price": 12.99,
        "quantity": 3
      }
    },
    "expected_result": null
  },
  {
